##Word Sense Disambiguation for Marathi Language-
Word Sense Disambiguation (WSD) is a natural language processing task of identifying the correct sense of a word in context. Many words in natural language have multiple senses, and it is often essential to disambiguate them to understand the meaning of the sentence accurately.

Required Library Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install pyconll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
! pip install pydot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pyconll

In [5]:
import nltk
import pandas as pd

from nltk import word_tokenize
from nltk.tag import untag
from nltk import UnigramTagger

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

from nltk.corpus import indian
nltk.download('indian')

<ipython-input-5-10b19ae55255>:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)
[nltk_data] Downloading package indian to /root/nltk_data...
[nltk_data]   Unzipping corpora/indian.zip.


True

In [6]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
TRAIN = '/content/drive/MyDrive/MArathi_data/mr_cmupan-ud-train.conllu'
DEV = '/content/drive/MyDrive/MArathi_data/mr_cmupan-ud-dev.conllu'
TEST = '/content/drive/MyDrive/MArathi_data/mr_cmupan-ud-test.conllu'

Taking 2 sample texts for testing

In [9]:
text = "हा लेख पन्ना जिल्ह्याविषयी आहे. पन्ना शहराविषयीचा लेख येथे आहे. पन्ना जिल्हा भारतातील मध्य प्रदेश राज्यातील एक जिल्हा आहे."
sent = "समाजासाठी देशासाठी चांगले काम करणाऱ्या व्यक्तीला लोक एखाद्या मानद उपाधीने संबोधित करायला सुरुवात करतात आणि पुढे तीच उपाधी त्या व्यक्तीची ओळख होऊन जाते. पुण्यश्लोक, महर्षी, महात्मा, बाबासाहेब आणि लोकमान्य या अशाच काही उपाध्या आहेत. भारतातील अशा उपाध्या धारण करणाऱ्या काही व्यक्तींची नावे पुढील पुढीलप्रमाणे आहेत."

##Preprocess

In [10]:
import re
# Remove unnecessary spaces
text = re.sub('\s+', ' ', text).strip()

# Remove numbers
text = re.sub('[0-9]+', '', text)

print(text)

हा लेख पन्ना जिल्ह्याविषयी आहे. पन्ना शहराविषयीचा लेख येथे आहे. पन्ना जिल्हा भारतातील मध्य प्रदेश राज्यातील एक जिल्हा आहे.


In [11]:
#Fuction to read universal POS repos ConLLU files
def read_conllu(path):
    data = pyconll.load_from_file(path)
    tagged_sentences=[]
    t=0
    for sentence in data:
        tagged_sentence=[]
        for token in sentence:
            if token.upos and token.form:
                t+=1
                tagged_sentence.append((token.form.lower(), token.upos))
        tagged_sentences.append(tagged_sentence)
    return tagged_sentences

In [12]:
#reading training, validation and test sentences
train_sentences = read_conllu(TRAIN)
val_sentences = read_conllu(DEV)
test_sentences = read_conllu(TEST)

In [13]:
def build_vocab(sentences):
    vocab =set()
    for sentence in sentences:
        for word in untag(sentence):
            vocab.add(word)
    return sorted(list(vocab))

In [14]:
words = build_vocab(train_sentences+val_sentences+test_sentences)

In [ ]:
words

##Parts of Speech Tagging

In [16]:
tagged_set = 'marathi.pos'
articles = indian.sents(tagged_set)

In [17]:
articles

[["''", 'सनातनवाद्यांनी', 'व', 'प्रतिगाम्यांनी', 'समाज', 'रसातळाला', 'नेला', 'असताना', 'या', 'अंधारात', 'बाळशास्त्री', 'जांभेकर', 'यांनी', "'दर्पण'च्या", 'माध्यमातून', 'पहिली', 'ज्ञानज्योत', 'तेववली', ',', "''", 'असे', 'प्रतिपादन', 'नटसम्राट', 'प्रभाकर', 'पणशीकर', 'यांनी', 'केले', '.'], ['दर्पणकार', 'बाळशास्त्री', 'जांभेकर', 'यांच्या', '१९५व्या', 'जयंतीनिमित्त', 'महाराष्ट्र', 'संपादक', 'परिषद', 'व', 'सिंधुदुर्ग', 'जिल्हा', 'मराठी', 'पत्रकार', 'संघाच्या', 'वतीने', 'तसेच', 'महाराष्ट्र', 'जर्नलिस्ट', 'फाउंडेशन', 'व', 'महाराष्ट्र', 'ग्रामीण', 'पत्रकार', 'संघाच्या', 'सहभागाने', 'अभिवादन', 'कार्यक्रम', 'आयोजित', 'केला', 'होता', '.'], ...]

In [18]:
w = []
for sent in articles:
  for word in sent:
    w.append(word)

In [20]:
trwv = words+w

In [44]:
count=len(articles)
print(count)

1197


In [45]:
sentence1 = " ".join(articles[0])
sentence1

"'' सनातनवाद्यांनी व प्रतिगाम्यांनी समाज रसातळाला नेला असताना या अंधारात बाळशास्त्री जांभेकर यांनी 'दर्पण'च्या माध्यमातून पहिली ज्ञानज्योत तेववली , '' असे प्रतिपादन नटसम्राट प्रभाकर पणशीकर यांनी केले ."

Making Training and testing set split

In [46]:
train_perc = .9
train_rows = int(train_perc*count)
test_rows = train_rows + 1
print(train_rows,test_rows)

1077 1078


Getting already tagged sentences for traning

In [47]:
data = indian.tagged_sents(tagged_set)


In [48]:
train_data = data[:train_rows] 
test_data = data[test_rows:]

##Traning Unigram POS Tagger

In [50]:
# Train the unigram model
unigram_tag = UnigramTagger(train_data)

# Test it on a single sentence
unigram_tag.tag(untag(test_data[0]))

[('ज्या', 'PRP'),
 ('देशवासीयांकडे', None),
 ('केबल', 'NN'),
 ('नाही', 'VAUX'),
 (',', 'SYM'),
 ('त्यांना', 'PRP'),
 ('सात', 'QC'),
 ('मिनिटे', 'NN'),
 ('उशिरा', None),
 ('सामना', 'NN'),
 ('दाखवावा', None),
 ('लागत', 'VAUX'),
 ('असल्याने', 'VM'),
 ('निंबस', 'NNP'),
 ('अन्याय', 'NN'),
 ('करत', 'VM'),
 ('असल्याचे', 'VAUX'),
 ('मत', 'NN'),
 ('त्यांनी', 'PRP'),
 ('व्यक्त', 'JJ'),
 ('केले', 'VM'),
 ('होते', 'VAUX'),
 (';', 'SYM'),
 ('तसेच', 'CC'),
 ('त्याच', 'DEM'),
 ('वेळी', 'NN'),
 ('सरकारी', 'JJ'),
 ('पातळीवर', 'NN'),
 ('एक', 'QC'),
 ('विधेयक', 'NN'),
 ('मांडण्याचाही', None),
 ('विचार', 'NN'),
 ('बोलून', 'VM'),
 ('दाखविला', 'VAUX'),
 ('होता', 'VAUX'),
 ('.', 'SYM')]

In [52]:
unigram_tag.evaluate(test_data)   # Evaluation of model

<ipython-input-52-d8fa693e3b9f>:1: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  unigram_tag.evaluate(test_data)   # Evaluation of model


0.5846774193548387

##Unigram Tagger with Default Backoff

In [55]:
default_tagger = nltk.DefaultTagger('NN')
unigram_tagger = nltk.UnigramTagger(train_sentences+val_sentences, backoff=default_tagger)


In [59]:
# Some usefull functions
def tag_sequence(sentences):
    return [[t for w, t in sentence] for sentence in sentences]

In [60]:
y_train = [item for sublist in tag_sequence(train_sentences+val_sentences) for item in sublist]
y_test = [item for sublist in tag_sequence(test_sentences) for item in sublist]
classes = sorted(list(set(y_train)))

In [61]:
def tag_list(tagged_sents):
    return [tag for sent in tagged_sents for (word, tag) in sent]
def apply_tagger(tagger, corpus):
    return [tagger.tag(nltk.tag.untag(sent)) for sent in corpus]

In [63]:
#import some useful metrics
from sklearn.metrics import recall_score, precision_score, classification_report, accuracy_score, confusion_matrix, f1_score

##77 percent

In [64]:
y_pred = tag_list(apply_tagger(unigram_tagger, test_sentences))
print("Accuracy: {0:.4f}".format(unigram_tagger.evaluate(test_sentences)))
print('f1-macro score: {0:.4f}'.format(f1_score(y_test, y_pred, average='macro')))

<ipython-input-64-07e75b121ec5>:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print("Accuracy: {0:.4f}".format(unigram_tagger.evaluate(test_sentences)))


Accuracy: 0.7752
f1-macro score: 0.6868


In [54]:
unigram_tagger.evaluate(test_data)  #Evaluation of new model

<ipython-input-54-1de5a2157274>:1: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  unigram_tagger.evaluate(test_data)  #Evaluation of new model


0.016129032258064516

##Function to get POS of any provided marathi text uisng the above Unigram Tagger

In [65]:
#Generalized function that would return the POS tags in a structured table format
from collections import defaultdict

def get_POS(article):

    POS = {}

    word_tags = unigram_tagger.tag(article)
    for word, tag in word_tags:
        if tag not in POS:
            POS[tag] = [word]
        else:
            POS[tag].append(word)
    
    DF = {'Tags':[], 'Words':[], 'Count':[]}


    for k in POS:
        DF['Tags'].append(k)
        DF['Words'].append(" ".join(POS[k]))
        DF['Count'].append(len(POS[k]))
    
    return pd.DataFrame(DF)

In [66]:
token = word_tokenize(text)

In [67]:
df = get_POS(token)

Conversion of POS to DataFrame

In [68]:
df.head(10)

,Tags,Words,Count
0,PRON,हा,1
1,NOUN,लेख लेख जिल्हा प्रदेश राज्यातील जिल्हा,6
2,NN,पन्ना जिल्ह्याविषयी पन्ना शहराविषयीचा पन्ना,5
3,VERB,आहे आहे आहे,3
4,PUNCT,. . .,3
5,NST,येथे,1
6,PROPN,भारतातील मध्य,2
7,NUM,एक,1


In [69]:
word_tags = unigram_tagger.tag(token)

In [70]:
word_tags

[('हा', 'PRON'),
 ('लेख', 'NOUN'),
 ('पन्ना', 'NN'),
 ('जिल्ह्याविषयी', 'NN'),
 ('आहे', 'VERB'),
 ('.', 'PUNCT'),
 ('पन्ना', 'NN'),
 ('शहराविषयीचा', 'NN'),
 ('लेख', 'NOUN'),
 ('येथे', 'NST'),
 ('आहे', 'VERB'),
 ('.', 'PUNCT'),
 ('पन्ना', 'NN'),
 ('जिल्हा', 'NOUN'),
 ('भारतातील', 'PROPN'),
 ('मध्य', 'PROPN'),
 ('प्रदेश', 'NOUN'),
 ('राज्यातील', 'NOUN'),
 ('एक', 'NUM'),
 ('जिल्हा', 'NOUN'),
 ('आहे', 'VERB'),
 ('.', 'PUNCT')]

##Word of Interest Lookup accroding to synset format

In [71]:
def wordOfInterest(pos):
    wn_pos=['NN','VB','JJ','JJR','JJS','NNP','VBG','RB','VBD','VBP','VAUX','PRP','DEM']

    woi1=[]
    for x in range(0,len(pos)):
      arr=[]
      j=pos[x]
      for y in range(0,len(j)):
        i=j[y]
        if i in wn_pos:
          arr.append(j)
          woi1.append(arr) 
    woi=[]


    for i in woi1:
        arr2=[]
        for j in i:

            if j[1]=='VBD' or j[1]=='VB' or j[1]=='VBP':
                tup=(j[0],'verb')
                arr2.append(tup)
            elif j[1]=='VBG' or j[1]=='PRP':
                tup=(j[0],'noun')
                arr2.append(tup)
            elif j[1]=='NN' or j[1]=='NNP':
                tup=(j[0],'noun')
                arr2.append(tup)
            elif j[1]== 'JJ' or j[1]=='JJR' or j[1]=='JJS' or j[1]=='VAUX' or j[1]=='DEM':
                tup=(j[0],'adjective')
                arr2.append(tup)
            elif j[1]=='RB' or j[1]=='VM':
                tup=(j[0],'adverb')
                arr2.append(tup)
        woi.append(arr2)       
            
    return woi

##Making a Wordnet dictionary

reading wordbank txt file and preprocessing

##Making Wordnet definitions - dictionary

In [72]:
def deleteLeadingZeros(inputString):
   # regex pattern for removing leading zeros from an input string
   regexPattern = "^0+(?!$)"
   # Replace the matched regex pattern with an empty string
   outputString = re.sub(regexPattern, "", inputString)
   # returning output string after removing leading 0s
   return outputString

In [73]:
# Initialize an empty dictionary
dictionary = {}

# Open the text file and read its contents line by line
with open("/content/drive/MyDrive/MArathi_data/data.txt", "r", encoding = 'utf-8') as f:
    for line in f:
        # Split the line into serial number and value
        line = line.strip()  # Remove any leading/trailing whitespace
        regexPattern = "^0+(?!$)"
        line = re.sub(regexPattern, "", line)
        serial_number, value = line.split(" ", 1)
        #serial_number = int(serial_number)  # Convert the serial number to an integer

        # Add the key-value pair to the dictionary
        dictionary[serial_number] = value

In [74]:
for n in dictionary.keys():
    # Remove numbers from the line using regular expressions
    line = dictionary.get(n)
    cleaned_line = re.sub(r'\d+', '', line)

    # Remove unnecessary spaces from the line
    cleaned_line = ' '.join(cleaned_line.strip().split())

    # Add the cleaned line to the list of cleaned lines
    dictionary.update({n:cleaned_line})

First 5 entries

In [75]:
count = 0
for n in dictionary.keys():
  count+=1
  if count<5:
    print(n, dictionary.get(n))
  else:
    break

1 अजन्मा | ज्यास जन्म नाही असा:"ईश्वर अजन्मा आहे"
2 अशुभ:अमंगळ | शुभ नाही असा:"या योगामूळे कुंडलीतील इतर अशुभ योगांचा नाश होतो."
3 अप्रविष्ट | ज्याने प्रवेश केला नाही असा:"अप्रविष्ट व्यक्तींना ताबडतोब आत प्रवेश करू द्या."
4 पुण्यभूमी:पवित्रभूमी:पुण्यस्थान:पवित्रस्थान:पावनस्थान | पवित्र मानले गेलेले स्थान:"हिंदूंसाठी काशी ही पुण्यभूमी आहे."


##Synsets 

In [76]:
!pip install --upgrade pyiwn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [77]:
import pyiwn

[██████████████████████████████████████████████████]


Marathi synset object creation

In [78]:
#marathi wordnet
iwn = pyiwn.IndoWordNet(lang=pyiwn.Language.MARATHI)

In [79]:
#checking if it works
all_synsets = iwn.synsets('आई')

In [80]:
all_synsets

[Synset('आई.noun.2191'), Synset('आई.noun.36505')]

List of tags available

In [81]:
#available tags
list(map(str, pyiwn.PosTag))

['PosTag.NOUN', 'PosTag.VERB', 'PosTag.ADVERB', 'PosTag.ADJECTIVE']

In [82]:
synsets = iwn.all_synsets()
print(len(synsets))

32200


In [83]:
syn = synsets[:5]

In [84]:
syn = str(synsets[:5])

In [85]:
syn = list(syn.split(","))

In [86]:
import string
idx = []
printable_chars = string.printable
for line in syn:
  cleaned_text = "".join([char for char in line if char in printable_chars and char not in string.whitespace])
  n = re.findall('\d+', cleaned_text)
  idx.append(n)

In [87]:
print(idx)

[['5'], ['1'], ['7'], ['2'], ['10']]


##Lookup

In [88]:
for n in idx:
  print(n, dictionary.get(n[0]))

['5'] शिवालय:शिवमंदिर | जिथे शंकराच्या पिंडीची स्थापना केली असून त्याची पूजा होते ती जागा:"ती दर सोमवारी शिवालयात जाते."
['1'] अजन्मा | ज्यास जन्म नाही असा:"ईश्वर अजन्मा आहे"
['7'] आलेला:आगत | दाखल झालेला:"आलेल्या पाहुण्यांचे स्वागत आहे."
['2'] अशुभ:अमंगळ | शुभ नाही असा:"या योगामूळे कुंडलीतील इतर अशुभ योगांचा नाश होतो."
['10'] उत्पादित:उत्पन्न | ज्याची उत्पत्ति झाली आहे असा:"आसाममध्ये उत्पादित चहा जगभर प्रसिद्ध आहे."


##Final Complete Function for WSD

In [89]:
text = str(input())


हा लेख पन्ना जिल्ह्याविषयी आहे. पन्ना शहराविषयीचा लेख येथे आहे. पन्ना जिल्हा भारतातील मध्य प्रदेश राज्यातील एक जिल्हा आहे


In [90]:
text

'हा लेख पन्ना जिल्ह्याविषयी आहे. पन्ना शहराविषयीचा लेख येथे आहे. पन्ना जिल्हा भारतातील मध्य प्रदेश राज्यातील एक जिल्हा आहे'

In [116]:
word = input()

प्रदेश


Function to Get Parts of Speech of the Input text

In [92]:
def get_pos(text):
  # Remove unnecessary spaces
  #text = re.sub('\s+', ' ', string).strip()

  # Remove numbers
  #text = re.sub('[0-9]+', '', string)
  
  #Parts of speech tagging
  POS = {}
  token = word_tokenize(text)
  
  word_tags = unigram_tagger.tag(token)
  
  for w, tag in word_tags:
    if tag not in POS:
      POS[tag] = [w]
    else:
      POS[tag].append(w)
  
  return POS,word_tags

DataFrame for better visuals

In [93]:
def pos_df(text):
  token = word_tokenize(text)
  df = get_POS(token)

  return df


Function to Get Word of Interest Tags

In [128]:
def get_word_of_interest(POS,wor):
  w=[]
  for token in POS:
    if(token[0] == wor):
      w.append(token[1])
    else:
      continue

  wn_pos=['NN','VB','JJ','JJR','JJS','NNP','VBG','RB','VBD','VBP','VAUX','PRP','DEM']
  for i in range(len(w)):
    if w[i]=='VBD' or w[i]=='VB' or w[i]=='VBP':
      w[i]='VERB'

    elif w[i]=='VBG' or w[i]=='PRP':
      w[i]='NOUN'

    elif w[i]=='NN' or w[i]=='NNP' or 'N' or 'NNC':
      w[i]='NOUN'

    elif w[i]== 'JJ' or w[i]=='JJR' or w[i]=='JJS' or w[i]=='VAUX' or w[i]=='DEM':
      w[i]='ADJECTIVE'

    elif w[i]=='RB' or w[i]=='VM':
      w[i]='ADVERB'

  return w

Function to get synsets of the word of Interest 

In [178]:
def get_synsets(w,wor):
  j = []  
  for i in range(len(w)):
    if len(w) == 1:
      if w[i] == "NOUN":
        j.append(str(iwn.synsets(wor, pos=pyiwn.PosTag.NOUN)))
        break
      elif w[i] == "ADJECTIVE":
      #all_synsets = iwn.synsets(word, pos=pyiwn.PosTag.ADJECTIVE)
        j.append(str(iwn.synsets(wor, pos=pyiwn.PosTag.ADJECTIVE)))
        break
      elif w[i] == "ADVERB":
      #all_synsets = iwn.synsets(word, pos=pyiwn.PosTag.ADVERB)
       j.append(str(iwn.synsets(wor, pos=pyiwn.PosTag.ADVERB)))
       break
      else:
      #all_synsets = iwn.synsets(word, pos=pyiwn.PosTag.VERB)
        j.append(str(iwn.synsets(wor, pos=pyiwn.PosTag.VERB)))
        break

    elif w[i] == w[i+1] == w[i+2]:
      if w[i] == "NOUN":
      #all_synsets = iwn.synsets(word, pos=pyiwn.PosTag.NOUN)
        j.append(str(iwn.synsets(wor, pos=pyiwn.PosTag.NOUN)))
        break
      elif w[i] == "ADJECTIVE":
      #all_synsets = iwn.synsets(word, pos=pyiwn.PosTag.ADJECTIVE)
        j.append(str(iwn.synsets(wor, pos=pyiwn.PosTag.ADJECTIVE)))
        break
      elif w[i] == "ADVERB":
      #all_synsets = iwn.synsets(word, pos=pyiwn.PosTag.ADVERB)
       j.append(str(iwn.synsets(wor, pos=pyiwn.PosTag.ADVERB)))
       break
      else:
      #all_synsets = iwn.synsets(word, pos=pyiwn.PosTag.VERB)
        j.append(str(iwn.synsets(wor, pos=pyiwn.PosTag.VERB)))
        break

  if len(j) !=0:
    all_synsets=[]
    for i in j:
      #for k in i:
      temp = i.split(",")
      all_synsets.append(temp)
  else:
    print("no synsets in dictionary")
  
  return all_synsets

Function to get the Index for Lookup

In [96]:
def get_index(synsets):
  idx = []
  
  #index idendification for lookup
  printable_chars = string.printable
  
  for line in synsets[0]:
    cleaned_text = "".join([char for char in line if char in printable_chars and char not in string.whitespace])
    n = re.findall('\d+', cleaned_text)
    idx.append(n)

  #returning the calculated results
  return idx

Get Parts of Speech Tags

In [129]:
#getting the parts od speech, synsets and index for the given text
pos, tags = get_pos(text)
print(pos)
print()
print(tags)

{'PRON': ['हा'], 'NOUN': ['लेख', 'लेख', 'जिल्हा', 'प्रदेश', 'राज्यातील', 'जिल्हा'], 'NN': ['पन्ना', 'जिल्ह्याविषयी', 'पन्ना', 'शहराविषयीचा', 'पन्ना'], 'VERB': ['आहे', 'आहे', 'आहे'], 'PUNCT': ['.', '.'], 'NST': ['येथे'], 'PROPN': ['भारतातील', 'मध्य'], 'NUM': ['एक']}

[('हा', 'PRON'), ('लेख', 'NOUN'), ('पन्ना', 'NN'), ('जिल्ह्याविषयी', 'NN'), ('आहे', 'VERB'), ('.', 'PUNCT'), ('पन्ना', 'NN'), ('शहराविषयीचा', 'NN'), ('लेख', 'NOUN'), ('येथे', 'NST'), ('आहे', 'VERB'), ('.', 'PUNCT'), ('पन्ना', 'NN'), ('जिल्हा', 'NOUN'), ('भारतातील', 'PROPN'), ('मध्य', 'PROPN'), ('प्रदेश', 'NOUN'), ('राज्यातील', 'NOUN'), ('एक', 'NUM'), ('जिल्हा', 'NOUN'), ('आहे', 'VERB')]


In [130]:
df_pos = pos_df(text)

Get word of Interest Tag

In [131]:
##corrected code
woi = get_word_of_interest(tags,word)
print(woi)

['NOUN']


Get synsets for the Word of Interest

In [141]:
synsets = get_synsets(woi,word)

In [142]:
print(synsets)

[["[Synset('राज्य.noun.231')", " Synset('प्रदेश.noun.2022')", " Synset('राज्य.noun.26705')]"]]


Get Index of the Word of Interest to Lookup in Dictionary

In [104]:
idx = get_index(synsets)
print(idx)

[['231'], ['2022'], ['26705']]


##Final Result

In [105]:
print("Parts of Speech Tags for the Given Text")
print(df_pos)
print()
print("Related synsets = ", "\n", synsets)
print()
print("Related indexes = ", "\n", idx)
print()
for n in idx:
  print("Word ID", n," - ", dictionary.get(n[0])) 

Parts of Speech Tags for the Given Text
    Tags                                        Words  Count
0  PRON   हा                                           1    
1  NOUN   लेख लेख जिल्हा प्रदेश राज्यातील जिल्हा       6    
2  NN     पन्ना जिल्ह्याविषयी पन्ना शहराविषयीचा पन्ना  5    
3  VERB   आहे आहे आहे                                  3    
4  PUNCT  . .                                          2    
5  NST    येथे                                         1    
6  PROPN  भारतातील मध्य                                2    
7  NUM    एक                                           1    

Related synsets =  
 [["[Synset('राज्य.noun.231')", " Synset('प्रदेश.noun.2022')", " Synset('राज्य.noun.26705')]"]]

Related indexes =  
 [['231'], ['2022'], ['26705']]

Word ID ['231']  -  प्रदेश:प्रांत:सुभा:मुलूख | विशिष्ट अशा मर्यादांनी निश्चित केलेला व ज्यात राहणार्‍या लोकांची भाषा, राहणीमान इत्यादी दुसर्‍या भागाच्या लोकांपेक्षा वेगळी असते असा देशाचा एक भाग:"प्रत्येक प्रदेशाच्या लोकांची राहणी वेगवेगळी अ

##Additional test cases

In [106]:
text1 = "समाजासाठी देशासाठी चांगले काम करणाऱ्या व्यक्तीला लोक एखाद्या मानद उपाधीने संबोधित करायला सुरुवात करतात आणि पुढे तीच उपाधी त्या व्यक्तीची ओळख होऊन जाते"
text2 = "पुण्यश्लोक, महर्षी, महात्मा, बाबासाहेब आणि लोकमान्य या अशाच काही उपाध्या आहेत."
text3 = "भारतातील अशा उपाध्या धारण करणाऱ्या काही व्यक्तींची नावे पुढील पुढीलप्रमाणे आहेत"
text4 = ""

In [179]:
word1 = "सुरुवात"
word2 = "महर्षी"
word3 = "धारण"

In [133]:
#getting the parts od speech, synsets and index for the given text
pos1, tags1 = get_pos(text1)
print(pos1)
print()
print(tags1)

{'NOUN': ['समाजासाठी', 'देशासाठी', 'काम', 'व्यक्तीला', 'लोक', 'सुरुवात', 'ओळख'], 'ADJ': ['चांगले', 'संबोधित'], 'VERB': ['करणाऱ्या', 'करायला', 'करतात', 'होऊन'], 'DET': ['एखाद्या'], 'NN': ['मानद', 'उपाधीने', 'उपाधी', 'व्यक्तीची'], 'SCONJ': ['आणि'], 'NST': ['पुढे'], 'PRON': ['तीच', 'त्या'], 'AUX': ['जाते']}

[('समाजासाठी', 'NOUN'), ('देशासाठी', 'NOUN'), ('चांगले', 'ADJ'), ('काम', 'NOUN'), ('करणाऱ्या', 'VERB'), ('व्यक्तीला', 'NOUN'), ('लोक', 'NOUN'), ('एखाद्या', 'DET'), ('मानद', 'NN'), ('उपाधीने', 'NN'), ('संबोधित', 'ADJ'), ('करायला', 'VERB'), ('सुरुवात', 'NOUN'), ('करतात', 'VERB'), ('आणि', 'SCONJ'), ('पुढे', 'NST'), ('तीच', 'PRON'), ('उपाधी', 'NN'), ('त्या', 'PRON'), ('व्यक्तीची', 'NN'), ('ओळख', 'NOUN'), ('होऊन', 'VERB'), ('जाते', 'AUX')]


In [134]:
df_pos = pos_df(text1)

In [157]:
woi = get_word_of_interest(tags1,word1)
print(woi)

['NOUN']


In [158]:
synsets = get_synsets(woi,word1)

In [159]:
print(synsets)

[["[Synset('सुरुवात.noun.67')]"]]


In [160]:
idx = get_index(synsets)
print(idx)

[['67']]


In [161]:
print("Parts of Speech Tags for the Given Text")
print(df_pos)
print()
print("Related synsets = ", "\n", synsets)
print()
print("Related indexes = ", "\n", idx)
print()
for n in idx:
  print("Word ID", n," - ", dictionary.get(n[0])) 

Parts of Speech Tags for the Given Text
    Tags                                             Words  Count
0  NOUN   समाजासाठी देशासाठी काम व्यक्तीला लोक सुरुवात ओळख  7    
1  ADJ    चांगले संबोधित                                    2    
2  VERB   करणाऱ्या करायला करतात होऊन                        4    
3  DET    एखाद्या                                           1    
4  NN     मानद उपाधीने उपाधी व्यक्तीची                      4    
5  SCONJ  आणि                                               1    
6  NST    पुढे                                              1    
7  PRON   तीच त्या                                          2    
8  AUX    जाते                                              1    

Related synsets =  
 [["[Synset('सुरुवात.noun.67')]"]]

Related indexes =  
 [['67']]

Word ID ['67']  -  सुरुवात:सुरवात:प्रारंभ:आरंभ:श्रीगणेशा:ओनामा:नांदी:मुहूर्त | एखादे कार्य वा गोष्ट सुरू होण्याची क्रिया:"सतराव्या शतकात इंग्लंडमध्ये औद्योगिक विकासाची सुरवात झाली."


#2 text

In [162]:
#getting the parts od speech, synsets and index for the given text
pos2, tags2 = get_pos(text2)
print(pos2)
print()
print(tags2)

{'NN': ['पुण्यश्लोक', 'उपाध्या'], 'PUNCT': [',', ',', ',', '.'], 'PROPN': ['महर्षी', 'महात्मा', 'बाबासाहेब', 'लोकमान्य'], 'SCONJ': ['आणि'], 'PRON': ['या', 'अशाच'], 'DET': ['काही'], 'VERB': ['आहेत']}

[('पुण्यश्लोक', 'NN'), (',', 'PUNCT'), ('महर्षी', 'PROPN'), (',', 'PUNCT'), ('महात्मा', 'PROPN'), (',', 'PUNCT'), ('बाबासाहेब', 'PROPN'), ('आणि', 'SCONJ'), ('लोकमान्य', 'PROPN'), ('या', 'PRON'), ('अशाच', 'PRON'), ('काही', 'DET'), ('उपाध्या', 'NN'), ('आहेत', 'VERB'), ('.', 'PUNCT')]


In [163]:
df_pos = pos_df(text2)

In [180]:
woi = get_word_of_interest(tags2,word2)
print(woi)

['NOUN']


In [181]:
synsets = get_synsets(woi,word2)

In [182]:
print(synsets)

[["[Synset('महर्षी.noun.17587')]"]]


In [183]:
idx = get_index(synsets)
print(idx)

[['17587']]


In [184]:
print("Parts of Speech Tags for the Given Text")
print(df_pos)
print()
print("Related synsets = ", "\n", synsets)
print()
print("Related indexes = ", "\n", idx)
print()
for n in idx:
  print("Word ID", n," - ", dictionary.get(n[0])) 

Parts of Speech Tags for the Given Text
    Tags                              Words  Count
0  NN     पुण्यश्लोक उपाध्या                 2    
1  PUNCT  , , , .                            4    
2  PROPN  महर्षी महात्मा बाबासाहेब लोकमान्य  4    
3  SCONJ  आणि                                1    
4  PRON   या अशाच                            2    
5  DET    काही                               1    
6  VERB   आहेत                               1    

Related synsets =  
 [["[Synset('महर्षी.noun.17587')]"]]

Related indexes =  
 [['17587']]

Word ID ['17587']  -  महर्षी | एक राग:"श्रोते संगीतज्ञाला महर्षी ऐकवण्यास आग्रह करत होते."


##text 3

In [185]:
#getting the parts od speech, synsets and index for the given text
pos3, tags3 = get_pos(text3)
print(pos3)
print()
print(tags3)

{'PROPN': ['भारतातील'], 'PRON': ['अशा'], 'NN': ['उपाध्या'], 'NOUN': ['धारण', 'व्यक्तींची', 'नावे'], 'VERB': ['करणाऱ्या', 'आहेत'], 'DET': ['काही'], 'NST': ['पुढील', 'पुढीलप्रमाणे']}

[('भारतातील', 'PROPN'), ('अशा', 'PRON'), ('उपाध्या', 'NN'), ('धारण', 'NOUN'), ('करणाऱ्या', 'VERB'), ('काही', 'DET'), ('व्यक्तींची', 'NOUN'), ('नावे', 'NOUN'), ('पुढील', 'NST'), ('पुढीलप्रमाणे', 'NST'), ('आहेत', 'VERB')]


In [186]:
df_pos = pos_df(text3)

In [187]:
woi = get_word_of_interest(tags3,word3)
print(woi)

['NOUN']


In [190]:
synsets = get_synsets(woi,word3)

In [191]:
print(synsets)

[["[Synset('परिधान.noun.20007')", " Synset('धारण.noun.20006')]"]]


In [192]:
idx = get_index(synsets)
print(idx)

[['20007'], ['20006']]


In [193]:
print("Parts of Speech Tags for the Given Text")
print(df_pos)
print()
print("Related synsets = ", "\n", synsets)
print()
print("Related indexes = ", "\n", idx)
print()
for n in idx:
  print("Word ID", n," - ", dictionary.get(n[0])) 

Parts of Speech Tags for the Given Text
    Tags                 Words  Count
0  PROPN  भारतातील              1    
1  PRON   अशा                   1    
2  NN     उपाध्या               1    
3  NOUN   धारण व्यक्तींची नावे  3    
4  VERB   करणाऱ्या आहेत         2    
5  DET    काही                  1    
6  NST    पुढील पुढीलप्रमाणे    2    

Related synsets =  
 [["[Synset('परिधान.noun.20007')", " Synset('धारण.noun.20006')]"]]

Related indexes =  
 [['20007'], ['20006']]

Word ID ['20007']  -  परिधान:धारण | नेसण्याची वा घालण्याची क्रिया:"महाराजांनी रेशमी वस्त्रे परिधान केली होती."
Word ID ['20006']  -  धारण | धरण्याची किंवा आपल्या अंगी बाळगण्याची क्रिया:"भगवान शंकारांनी गंगेला आपल्या मस्तकी धारण केले आहे."
